### This Notebook will use to classifies Flower using a Pre trained Network

I'm using Vgg16 for the proposed task but you can modify the code accordingly

In [ ]:
#Import Libraries
import numpy as np
import torch
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Import useful funcions
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler

#Define your tranformer "How do you want your data"
#Oh!! Yea 
transform = transforms.Compose([transforms.RandomResizedCrop(224),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5),
                                                    (0.5, 0.5, 0.5))
                               ])
#Define your Data directory
data_dir = ('flower_photos/')
train_dir = os.path.join(data_dir, 'train/' )
test_dir = os.path.join(data_dir, 'test/')

#Define your Validation set
valid_size = 0.2
batch_size = 32


#import your Data using datasets 'ImageFloder'
train_set = datasets.ImageFolder(train_dir ,transform = transform)
test_set = datasets.ImageFolder(test_dir, transform = transform)

#Define your validation test

num_train = len(train_set)
indices = (list(range(num_train)))
np.random.shuffle(indices)
split = int(np.squeeze(valid_size * num_train))
train_dx , valid_dx = indices[split:], indices [:split]

#Now use from torch.utils.data.sampler import SubsetRandomSampler
train_sampler = SubsetRandomSampler(train_dx)
valid_sampler = SubsetRandomSampler(valid_dx)

# Define you data sets using 'DataLoader'

trainloader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, sampler = train_sampler, num_workers = 0)
vaildloader = torch.utils.data.DataLoader(train_set, batch_size = batch_size, sampler = valid_sampler, num_workers = 0)
testloader = torch.utils.data.DataLoader(test_set, batch_size = batch_size, shuffle = True, num_workers = 0)

In [3]:
#Take a look at Data sets
print( "Length of Training set is :{}".format(len(train_set)))
print ("Length of Test set is : {}".format(len(test_set)))

Length of Training set is :3130
Length of Test set is : 540


## Define your Model.

I'm using Models module from PyTorch to call Vgg16 there are lot more pretrained model. Please refer to the official documentation for more info.

In [4]:
#Define your model for the current task I used VGG16

Vgg16 = models.vgg16(pretrained=True)

#view Your model

print( Vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [5]:
print(Vgg16.classifier[6].in_features)
print(Vgg16.classifier[6].out_features)

4096
1000


### Freeze the model parameters

In [6]:
for params in Vgg16.features.parameters():
    params.requiues_grad = False

### Make changes to exiting model accroding to your requirement

In [7]:
from collections import OrderedDict
# Hyperparameters
input_layer = Vgg16.classifier[6].in_features # NO Confusion!!! :D
output_layer = 5

#Define you layer
classifier = nn.Linear(input_layer, output_layer)
                                   
# Commit the changes
Vgg16.classifier[6] = classifier


In [8]:
print (Vgg16) 
# Take a look at the model
# OR Use
#print (Vgg16.classifier[6].out_features)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Define Loss and Optimizer 

In [12]:
#Define the Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()

#Hyperparameters
Learning_rate = lr = 0.001
#Define the Optimizer
from torch import optim

optimizer = optim.SGD(Vgg16.classifier.parameters(), Learning_rate, momentum= 0.3)

### Train the Model
Oh!!! Yea

In [ ]:
# Hyperparameters
n_epochs = 8

valid_loss_min = np.Inf
images, labels = next(iter(trainloader))

for epochs in range(n_epochs):
    train_loss = 0
    valid_loss = 0
    Vgg16.train()
    for images, labels in trainloader:
        optimizer.zero_grad()
        output = Vgg16(images)
        loss = criterion(output, labels)
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    Vgg16.eval()
    for images, labels in validloader:
        output_valid = Vgg16(images)
        loss = criterion(output_valid, labels)
        
        valid_loss += loss.item()
    train_loss = train_loss/len(trainloader.sampler)
    valid_loss = valid_loss/len(validloader.sampler)
    
    print ('Epochs: {} \tTraining Loss: {:.6f} \tVaildation Loss: {:.6f}'.format(epochs+1, train_loss, valid_loss))
    
    #Save the model if Validation loss decreases
    if valid_loss < valid_loss_min:
        print ('Validation loss decreased ({:.6f} ---> {:.6f}. Saving Model...)'.format(valid_loss_min, valid_loss))
        torch.save(Vgg16.state_dict(), 'model.pt')
        valid_loss_min = valid_loss